In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.vectorstores.utils import filter_documents
from dotenv import load_dotenv

import json
import os

In [5]:
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [6]:
docs = [
    "Tesla released Model Y in 2024 and it saw huge demand in Europe.",
    "Apple introduced the M3 chip in 2023 with improved efficiency.",
    "In 2022, Google launched the Pixel 7 with better AI features."
]

In [7]:
llm = ChatOpenAI(temperature=0)

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_10902/3681847723.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0)


LLM for Tagging

In [8]:
tag_prompt = PromptTemplate.from_template("""
Extract metadata from this text.

Text:
{text}

Return as JSON with keys: company, product, year.
""")

In [9]:
def tag_document(text):
    response = llm.invoke(tag_prompt.format(text=text))
    return json.loads(response.content)

Convert Documents with tags

In [10]:
tagged_docs = []
for d in docs:
    tags = tag_document(d)
    tagged_docs.append(Document(page_content=d, metadata=tags))

In [11]:
tagged_docs

[Document(metadata={'company': 'Tesla', 'product': 'Model Y', 'year': 2024}, page_content='Tesla released Model Y in 2024 and it saw huge demand in Europe.'),
 Document(metadata={'company': 'Apple', 'product': 'M3 chip', 'year': 2023}, page_content='Apple introduced the M3 chip in 2023 with improved efficiency.'),
 Document(metadata={'company': 'Google', 'product': 'Pixel 7', 'year': 2022}, page_content='In 2022, Google launched the Pixel 7 with better AI features.')]

Split and Embedd Documents

In [12]:
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = splitter.split_documents(tagged_docs)

embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(chunks, embedding, collection_name="rag-tagged")

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_10902/1689448507.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding = OpenAIEmbeddings()


Extract info from query

In [13]:
query = "What did Tesla launch in 2024?"

query_tag_prompt = PromptTemplate.from_template("""
Extract company and year from this question:

Question:
{query}

Return as JSON with: company, year
""")

In [14]:
query_tags = llm.invoke(query_tag_prompt.format(query=query))
query_info = json.loads(query_tags.content)

In [15]:
query_info

{'company': 'Tesla', 'year': '2024'}

Filter and retrieve

In [ ]:
retriever = vectordb.as_retriever(search_kwargs={
    "filter": {
        "$and":[
                "company": query_info["company"],
                "year": query_info["year"]
        ]
    }
})

Ask final question with Retrieved documents


In [17]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

In [18]:
response = qa_chain("What did Tesla launch in 2024?")
print("Answer:", response["result"])

/var/folders/s8/qyjb36g92fs3ztdqk120mmkw0000gn/T/ipykernel_10902/4277824939.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain("What did Tesla launch in 2024?")


ValueError: Expected where to have exactly one operator, got {'company': 'Tesla', 'year': '2024'} in query.